In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya
model_base = malaya.segmentation.transformer(model = 'base')

In [3]:
import json

with open('prepare/test-set-segmentation.json') as fopen:
    data = json.load(fopen)
    
X, Y = [], []
for x, y in data:
    X.append(x)
    Y.append(y)

In [4]:
model_base.greedy_decoder(X[:10])

['Nihonbashi Mitsui Tower. Menara Nihonbashi Mitsui (Ri Ben Qiao San Jing tawa, "Nihonbashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo, Tokyo, Jepun. Dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. Pembinaannya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen Mitsukoshimae, Tokyo Metro.',
 'nihonbashi mitsui tower. menara nihonbashi mitsui (ri ben qiao san jing tawa, "nihonbashi mtusi tawa") merupakan sebuah bangunan pencakar langit yang terletak di chuo, tokyo, jepun. dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. pembinaannya telah siap pada tahun 2005. ia dihubungkan oleh stesen bawah tanah, stesen mitsukoshimae, tokyo metro.',
 'Nihonbashi Mitsui Tower. Menara Nihonbashi Mitsui (Ri Ben Qiao San Jing tawa, "Nihonbashi Mtusi Tawa") merupakan sebuah bangunan pen

In [5]:
from tqdm import tqdm

batch_size = 10
results = []
for i in tqdm(range(0, len(X), batch_size)):
    batch_x = X[i: i + batch_size]
    r = model_base.greedy_decoder(batch_x)
    results.extend(r)

100%|██████████| 9939/9939 [2:46:53<00:00,  1.01s/it]   


In [10]:
results_Y = results[:]
results_Y[0], Y[0]

('Nihonbashi Mitsui Tower. Menara Nihonbashi Mitsui (Ri Ben Qiao San Jing tawa, "Nihonbashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo, Tokyo, Jepun. Dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. Pembinaannya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen Mitsukoshimae, Tokyo Metro.',
 'Nihonbashi Mitsui Tower. Menara Nihonbashi Mitsui (Ri Ben Qiao San Jing tawa, "Nihonbashi Mtusi Tawa") merupakan sebuah bangunan pencakar langit yang terletak di Chuo, Tokyo, Jepun. Dengan ketinggian 192 meter, bangunan 34 tingkat ini digunakan terutamanya untuk ruang pejabat dan hotel di bahagian tingkat atas. Pembinaannya telah siap pada tahun 2005. Ia dihubungkan oleh stesen bawah tanah, Stesen Mitsukoshimae, Tokyo Metro.')

In [11]:
def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [12]:
wer = []
for i in tqdm(range(len(results_Y))):
    wer.append(calculate_wer(Y[i], results_Y[i]))

100%|██████████| 99384/99384 [00:03<00:00, 32582.07it/s]


In [13]:
import numpy as np
np.mean(wer)

0.17762361859977974